In [62]:
import csv
import pprint
import re

from os import listdir

import pdfplumber

HEADERS = (
    'work_order',
    'service_code',
    'service_address',
    'latitude',
    'longitude',
    'an_action_date',
    'an_status',
    'an_action_by',
    'an_action_notes',
    'rs_activity',
    'rs_type',
    'rs_resource',
    'rs_qty',
    'rs_units',
    'rs_performed_on',
)

# get the file name list
pdf_files = listdir('pdf')
# for dev, just first three
# pdf_files = pdf_files[:5]
# print(pdf_files)

masterlist = []

for file_name in pdf_files:
    
    with pdfplumber.open('pdf/{}'.format(file_name)) as pdf:
        
        pages = pdf.pages[0]

        lines = pages.extract_text()
        
        lines = enumerate(lines.split('\n'))
        iter_lines = iter(lines)

        for line_id, line in iter_lines:
            # get Work Order ID
            if line.startswith('Work Order ID'):
                work_order_id_plus = line.split('Work Order ID: ')[1]
                work_order_id = work_order_id_plus.split(' Work Order Date: ')[0]
    #             print(work_order_id)

            # get Service Code
            if line.startswith('Service Code'):
                service_code, = re.findall(r'\[([^\]]*)\]', line)
                
            # get Service Address
            if line.startswith('Service Address'):
                index, service_address = next(iter_lines)
                
            # get Latitude, Longitude
            if line.startswith('Latitude'):
                index, lat_lng_string = next(iter_lines)
                lat, lng = lat_lng_string.split()
            
#             print(line_id, line) # to see top lines

        # get the Action Notes & Result Summary tables
        try:
            action_notes, result_summary, activity_notes = pages.extract_tables()
#             pprint.pprint(action_notes)
#             pprint.pprint(result_summary)
#             pprint.pprint(activity_notes)
        
            # get the Action Notes
            
            # get the Result Summary
        except ValueError:
            pass
#             print(">>> BAD PDF: {}, '{}', '{}', {} {}".format(work_order_id, service_code, service_address, lat, lng), '\n\n')

        masterlist.append([int(work_order_id), service_code, service_address, float(lat), float(lng)])
        print('{}, "{}", "{}", {} {}'.format(work_order_id, service_code, service_address, lat, lng))

with open("working.csv", "w") as outfile:
    writer = csv.writer(outfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(HEADERS)
    for row in masterlist:
        writer.writerow(row)
    
#     im = p0.to_image()
#     im.debug_tablefinder()
#     im.save('/Users/jpheasly/Development/homeless-cleanups/foo.png', format="PNG")
    